An example of searching for the closest value(for a field) in elasticsearch

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.client import CatClient
from elasticsearch.helpers import bulk

In [2]:
elastic_db = Elasticsearch(['localhost:9200'])
cat_client = CatClient(elastic_db)

In [3]:
# To view the indices on Elastisearch:
print(cat_client.indices(v=True))

health status index   uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   .kibana _RRwYu7rSd2H7CuBLtZ4dw   1   1          2            1     12.2kb         12.2kb



In [4]:
# From 
# https://elasticsearch-py.readthedocs.io/en/master/helpers.html#bulk-helpers
# All bulk helpers accept an instance of Elasticsearch class 
# and an iterable actions (any iterable, can also be a generator,
# which is ideal in most cases since it will allow you to index 
# large datasets without the need of loading them into memory).
def gendata():
    values = [1,3,5,7,9]
    for val in values:
        yield {
            "_index": "closest_val",
            "_type": "doc",
            "value": val
        }
    # end for

In [5]:
response = bulk(client=elastic_db, actions=gendata(),stats_only=False)
print(response)

(5, [])


In [6]:
# To view the indices on Elastisearch:
print(cat_client.indices(v=True))

health status index       uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   closest_val yiaV2NUJQ4aR9edzqqqlSA   5   1          5            0      2.8kb          2.8kb
yellow open   .kibana     _RRwYu7rSd2H7CuBLtZ4dw   1   1          2            1     12.2kb         12.2kb



In [7]:
# Query. Change "val" to pass number of your choice
sch_body = {
    "query": {
        "function_score": {
            "query": { "match_all": {} },
            "script_score" : {
                "script" : {
                    "params": { "val": 1 },
                    "source": "-(Math.abs(params.val - doc['value'].value))"
                }
            },
            "boost_mode": "replace"
        }
    }
}

response = elastic_db.search(index='closest_val', body=sch_body, explain=False)
print(response['hits']['hits'][0])

{'_index': 'closest_val', '_source': {'value': 1}, '_type': 'doc', '_score': -0.0, '_id': 'an3EQWUBOrnFUpGSLhXv'}
